In [1]:
import larix as lx
# import jax
# import jax.numpy as jnp
from pytest import approx

/Users/jeffnewman/LocalGit/larix/larix/model/numbamodel.py:23: UserWarning: 

##### larix is experimental, and not feature-complete  #####
the first time you import on a new system, this package will
compile optimized binaries for your machine, which may take 
a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
d = lx.examples.MTC(format='dataset')

In [3]:
d['chose']

<xarray.DataArray 'chose' (caseid: 5029, altid: 6)>
array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       ...,
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)
Coordinates:
  * caseid    (caseid) int64 1 2 3 4 5 6 7 ... 5024 5025 5026 5027 5028 5029
  * altid     (altid) int64 1 2 3 4 5 6
    altnames  (altid) <U7 'DA' 'SR2' 'SR3+' 'Transit' 'Bike' 'Walk'

In [4]:
from xarray.core.extensions import AccessorRegistrationWarning

In [5]:
import warnings
warnings.filterwarnings("error", category=AccessorRegistrationWarning)


In [6]:
m = lx.Model(d)
P, X, PX = lx.P, lx.X, lx.PX
m.utility_co[2] = P("ASC_SR2")  + P("hhinc#2") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

m.utility_ca = PX("tottime") + PX("totcost")

m.availability_var = 'avail'
m.choice_ca_var = 'chose'

m.title = "MTC Example 1 (Simple MNL)"
r = m.maximize_loglike()

,value,best,initvalue,minimum,maximum,nullvalue
param_name,,,,,,
ASC_BIKE,-2.376328,-2.376328,0.0,-inf,inf,0.0
ASC_SR2,-2.178014,-2.178014,0.0,-inf,inf,0.0
ASC_SR3P,-3.725078,-3.725078,0.0,-inf,inf,0.0
ASC_TRAN,-0.670861,-0.670861,0.0,-inf,inf,0.0
ASC_WALK,-0.206775,-0.206775,0.0,-inf,inf,0.0
hhinc#2,-0.002170,-0.002170,0.0,-inf,inf,0.0
hhinc#3,0.000358,0.000358,0.0,-inf,inf,0.0
hhinc#4,-0.005286,-0.005286,0.0,-inf,inf,0.0
hhinc#5,-0.012808,-0.012808,0.0,-inf,inf,0.0


In [7]:
m.pf

,value,best,initvalue,minimum,maximum,nullvalue
param_name,,,,,,
ASC_BIKE,-2.376328,-2.376328,0.0,-inf,inf,0.0
ASC_SR2,-2.178014,-2.178014,0.0,-inf,inf,0.0
ASC_SR3P,-3.725078,-3.725078,0.0,-inf,inf,0.0
ASC_TRAN,-0.670861,-0.670861,0.0,-inf,inf,0.0
ASC_WALK,-0.206775,-0.206775,0.0,-inf,inf,0.0
hhinc#2,-0.002170,-0.002170,0.0,-inf,inf,0.0
hhinc#3,0.000358,0.000358,0.0,-inf,inf,0.0
hhinc#4,-0.005286,-0.005286,0.0,-inf,inf,0.0
hhinc#5,-0.012808,-0.012808,0.0,-inf,inf,0.0


In [8]:
m.loglike(m.pvals)

-3626.1862555129305

In [9]:
m.d_loglike()

array([ 8.630664e-04, -3.820780e-04, -1.382992e-03, -1.452203e-02,  3.967864e-03, -4.204360e-02, -9.950474e-02,
       -7.322533e-01,  3.308188e-02,  1.695009e-01, -2.320521e+00, -2.725375e-02])

In [10]:
m.required_data()

key,value
ca,"['totcost', 'tottime']"
co,['hhinc']
choice_ca,'chose'
avail_ca,'avail'


In [21]:
import jax.numpy as jnp

def _get_jnp_array(dataset, name):
    if name not in dataset:
        return None
    return jnp.asarray(dataset[name])


data_bundle = (
    _get_jnp_array(m.dataset, "ca"),
    _get_jnp_array(m.dataset, "co"),
    _get_jnp_array(m.dataset, "ch"),
    _get_jnp_array(m.dataset, "av"),
    _get_jnp_array(m.dataset, "wt"),
)

#m._jax_utility_bundle(m.pvals, data_bundle )

In [22]:
import jax
jax.vmap(m._jax_utility_bundle, in_axes=(None, 0))(m.pvals, data_bundle )

ValueError: too many values to unpack (expected 4)

In [11]:
m.jax_loglike(m.pvals)

DeviceArray(-3626.186, dtype=float32)

In [24]:
m.jax_loglike(m.pvals) # 622ms

DeviceArray(-3626.186, dtype=float32)

In [25]:
m.jax_loglike(m.pvals) # 622ms

DeviceArray(-3626.186, dtype=float32)

In [26]:
r = m.jax_maximize_loglike()

In [27]:
assert m.jax_loglike(m.pvals) == approx(-3626.186)

In [28]:
m.jax_d_loglike(m.pvals) # 1.69s

DeviceArray([ 8.428097e-04, -5.550385e-04, -1.508236e-03, -1.456070e-02,  3.928900e-03, -5.284785e-02, -1.080932e-01,
             -7.334058e-01,  3.241288e-02,  1.675611e-01, -2.169489e+00, -2.884785e-02], dtype=float32)

In [31]:
pocket = np.asarray([
    -1.188164e+00, -1.089007e+00, -1.862539e+00, -3.354305e-01, -1.033876e-01, -1.084969e-03,  1.788532e-04,
       -2.643163e-03, -6.403988e-03, -4.843151e-03, -2.460121e-03, -2.567105e-02])

In [36]:
assert m.d_loglike(pocket) == approx([  
    -100.668873,     -427.881921,   -346.047421,   
    -176.396743,      -69.950285, -25639.434015, 
    -20581.626877, -10752.088732,  -5810.403543,  
    -4360.946795,   54115.923707, -16552.567557,
], rel=1e-5)

In [37]:
assert m.jax_d_loglike(pocket) == approx([  
    -100.668873,     -427.881921,   -346.047421,   
    -176.396743,      -69.950285, -25639.434015, 
    -20581.626877, -10752.088732,  -5810.403543,  
    -4360.946795,   54115.923707, -16552.567557,
], rel=1e-5)

In [38]:
m.parameters

<xarray.Dataset>
Dimensions:     (param_name: 12)
Coordinates:
  * param_name  (param_name) <U8 'ASC_BIKE' 'ASC_SR2' ... 'totcost' 'tottime'
Data variables:
    value       (param_name) float64 -1.188 -1.089 -1.863 ... -0.00246 -0.02567
    initvalue   (param_name) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    nullvalue   (param_name) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    minimum     (param_name) float32 -inf -inf -inf -inf ... -inf -inf -inf -inf
    maximum     (param_name) float32 inf inf inf inf inf ... inf inf inf inf inf
    holdfast    (param_name) int8 0 0 0 0 0 0 0 0 0 0 0 0
    best        (param_name) float64 -2.376 -2.178 -3.725 ... -0.00492 -0.05134

In [39]:
m.pvals = 0

In [41]:
rj = m.jax_maximize_loglike() # 263ms

In [42]:
rj

     jac: array([ 2.272785e-02, -3.063679e-02, -8.840561e-03,  1.739502e-03, -6.566763e-03, -1.223307e+00, -1.308636e-01,
       -7.242671e-01,  1.559882e+00,  9.642785e-01, -5.636771e+00, -6.563418e-01])
 loglike: -3626.18603515625
 message: 'Optimization terminated successfully'
    nfev: 119
     nit: 39
    njev: 39
  status: 0
 success: True
       x: array([-2.376162e+00, -2.177859e+00, -3.724838e+00, -6.710735e-01, -2.063628e-01, -2.171617e-03,  3.547370e-04,
       -5.285348e-03, -1.282182e-02, -9.697886e-03, -4.919956e-03, -5.133474e-02])

In [43]:
assert rj.loglike == approx(-3626.18603515625)

In [44]:
h = m.jax_d2_loglike(m.pvals) # 4.00s

In [45]:
np.sqrt(np.linalg.pinv(h).diagonal())

array([ 3.045759e-01,  1.046350e-01,  1.776854e-01,  1.325887e-01,  1.941035e-01,  1.553271e-03,  2.537732e-03,
        1.828749e-03,  5.326436e-03,  3.033601e-03,  2.388836e-04,  3.099270e-03], dtype=float32)

In [46]:
ihess = m.jax_invhess_loglike(m.pvals) # 2.65s

In [47]:
se, hess, ihess = m.jax_param_cov(m.pvals) # 199ms

In [48]:
assert m.pstderr == approx(np.array([ 3.045759e-01,  1.046350e-01,  1.776854e-01,  1.325887e-01,  1.941035e-01,  1.553271e-03,  2.537731e-03,
        1.828749e-03,  5.326436e-03,  3.033601e-03,  2.388836e-04,  3.099270e-03], dtype=np.float32))